# Simple PyGDF Example

Redo the Simple Numba example to use PyGDF
This Example uses a smaller dataset and combines a number of the functions into a single operation

In [1]:
# This automatically time every cell's execution
!pip install ipython-autotime
%load_ext autotime

You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
#Imports 
import numpy as np 
import pandas as pd
import math

import pygdf 
import numba as nb
from numba import vectorize, jit

time: 345 ms


In [3]:
# Read in data and adjust
pdf = pd.read_csv("./data/1B.csv")
pdf.shape

(134217728, 1)

time: 14 s


In [4]:
pdf.size

134217728

time: 4.65 ms


In [5]:
# Convert to a GPU DataFrame
gdf = pygdf.DataFrame.from_pandas(pdf)

time: 678 ms


### Scale the data so that it is in the range of -200 to +200
This will also create a new column name

In [6]:
# Scale and Converet the data
def ScaleData( Data, B, Scale) :
    for i in range (Data.size) :
        B[i] = Data[i] * Scale

time: 2.07 ms


In [7]:
gdf = gdf.apply_rows(ScaleData, incols=['Data'], outcols=dict(B=np.float32), kwargs=dict(Scale=200))

time: 196 ms


In [8]:
gdf.head()

        Data        B
0  -0.293219 -58.6438
1   0.253161  50.6322
2 -0.0570855 -11.4171
3  0.0845902   16.918
4   -1.43822 -287.644

time: 7.58 ms


In [9]:
gdf.drop_column('Data')

time: 1.66 ms


In [10]:
gdf.head()

         B
0 -58.6438
1  50.6322
2 -11.4171
3   16.918
4 -287.644

time: 7.2 ms


### Compute the Mean and Standard Deviation

In [11]:
mean = gdf['B'].mean()
std = gdf['B'].std()
print("Std == %f \t Mean == %f" % (std, mean))

Std == 199.964919 	 Mean == 0.059651
time: 5.58 s


### Normalize

In [12]:
def NormalizeData( B, C, s, m) :
    for i in range(B.size):
        C[i] = math.floor((B[i] - m)/s)

time: 2.17 ms


In [13]:
gdf = gdf.apply_rows(NormalizeData, incols=['B'], outcols=dict(C=np.float32), kwargs=dict(s=std, m=mean ))

time: 195 ms


In [14]:
gdf.drop_column('B')
gdf.head()

     C
0 -1.0
1  0.0
2 -1.0
3  0.0
4 -2.0

time: 7.25 ms


# Group and count

In [15]:
from collections import OrderedDict

aggs = OrderedDict()
aggs['C'] = 'count'

stats = gdf.groupby(['C']).agg(aggs)


time: 1.18 s


In [16]:
stats

            C
 0       80.0
 1     3580.0
 2   176120.0
 3  2880680.0
 4 18230823.0
 5 45826641.0
 6 45797164.0
 7 18259060.0
 8  2865860.0
 9   173760.0
10     3940.0
11       20.0

time: 3.66 ms
